<a href="https://colab.research.google.com/github/isabellacadisco/Algorithms-for-Massive-Data-course/blob/main/LSH3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [134]:
import os

In [135]:
# download dataset from kaggle and save it in drive


#import os
os.environ['KAGGLE_USERNAME'] = "isabellacadisco"
os.environ['KAGGLE_KEY'] = "847325691c2399e41dc9c0b78fbe499e"
!kaggle datasets download -d xhlulu/medal-emnlp

 66% 4.49G/6.82G [01:02<00:32, 76.6MB/s]
User cancelled operation


In [56]:
!unzip medal-emnlp.zip #unzip dataset

Archive:  medal-emnlp.zip
replace full_data.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace pretrain_subset/test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace pretrain_subset/train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace pretrain_subset/valid.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [136]:
# set up spark
!apt-get install openjdk-8-jdk-headless -qq > /dev/null #install java as VM
!wget -q https://dlcdn.apache.org/spark/spark-3.4.0/spark-3.4.0-bin-hadoop3.tgz #download Spark
!tar xf spark-3.4.0-bin-hadoop3.tgz #unzip the file
!pip install -q findspark

In [137]:
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.4.0-bin-hadoop3"

In [138]:
import findspark
findspark.init("spark-3.4.0-bin-hadoop3")# SPARK_HOME
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate() #SPARK SESSION

In [139]:
import pyspark
type(spark)

pyspark.sql.session.SparkSession

In [140]:
file_path = './full_data.csv'

In [141]:
data_df = spark.read.csv(file_path, header=True)

In [142]:
data_df.show()

+--------------------+--------------------+--------------------+
|                TEXT|            LOCATION|               LABEL|
+--------------------+--------------------+--------------------+
|alphabisabolol ha...|                  56|           substrate|
|a report is given...|24|49|68|113|137|172|carcinosarcoma|re...|
|the virostatic co...|                  55|           substrate|
|rmi rmi and rmi a...|   25|82|127|182|222|compounds|compoun...|
|a doubleblind stu...|22|26|28|77|90|14...|oxazepam|placebo|...|
|stroma from eithe...|         6|82|84|107|red cells|serum|a...|
|the effect of the...|                4|13|major|pentose pho...|
|in one experiment...|        32|44|76|135|feeding|feeding|a...|
|the presence of a...|7|15|63|137|199|2...|active|study|acti...|
|the reaction of g...|     113|203|209|250|stable|assay|bind...|
|choline acetyltra...|                  44|             caudate|
|increasing concen...|                  81|        displacement|
|the properties of...|   

In [143]:
from pyspark.sql.functions import monotonically_increasing_id

In [144]:
dataset = data_df.select('TEXT').withColumn('doc_id', monotonically_increasing_id())

In [145]:
dataset.show()

+--------------------+------+
|                TEXT|doc_id|
+--------------------+------+
|alphabisabolol ha...|     0|
|a report is given...|     1|
|the virostatic co...|     2|
|rmi rmi and rmi a...|     3|
|a doubleblind stu...|     4|
|stroma from eithe...|     5|
|the effect of the...|     6|
|in one experiment...|     7|
|the presence of a...|     8|
|the reaction of g...|     9|
|choline acetyltra...|    10|
|increasing concen...|    11|
|the properties of...|    12|
|primary amines re...|    13|
|a purification pr...|    14|
|dihydrofolate red...|    15|
|ionization effect...|    16|
|kinetic analyses ...|    17|
|the nearultraviol...|    18|
|the circular pola...|    19|
+--------------------+------+
only showing top 20 rows



In [146]:
sampled_dataset = dataset.sample(withReplacement=False, fraction=0.00007, seed=42)

In [147]:
sampled_dataset.count()

935

In [148]:
sampled_dataset.show()

+--------------------+----------+
|                TEXT|    doc_id|
+--------------------+----------+
|guinea worm disea...|     54857|
|clinically uninvo...|     60617|
|the clinical stru...|     66206|
|in a lot of patie...|     71617|
|human term placen...|     72115|
|a recent outbreak...|    106539|
|qualitative and q...|    117032|
|four patients wit...|    117862|
|as life expectanc...|    125607|
|the purpose of th...|    125621|
|gastrointestinal ...|    129527|
|previous experime...|8589974423|
|mice received rad...|8589977320|
|food intake and r...|8589989283|
|the phenomenon of...|8590016766|
|the possibility o...|8590033288|
|a method is descr...|8590041248|
|the effect of int...|8590045842|
|muscle biopsy tis...|8590063333|
|approximately pre...|8590069516|
+--------------------+----------+
only showing top 20 rows



# Class Text Processor

In [149]:
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [150]:
nltk.download("punkt")
nltk.download("stopwords")
nltk.download("wordnet")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [151]:
class TextPreprocessor:
    def __init__(self):
        # Initialize NLTK resources
        #nltk.download("punkt")
        #nltk.download("stopwords")
        #nltk.download("wordnet")
        self.stop_words = set(stopwords.words("english"))
        self.lemmatizer = WordNetLemmatizer()

    # get position of words to lemmatize, info needed
    def get_word_pos(self, word):
        tagged_words = nltk.pos_tag([word])
        pos_tag = tagged_words[0][1]

        if pos_tag.startswith('J'):
            return wordnet.ADJ
        elif pos_tag.startswith('V'):
            return wordnet.VERB
        elif pos_tag.startswith('N'):
            return wordnet.NOUN
        elif pos_tag.startswith('R'):
            return wordnet.ADV
        else:
            return wordnet.NOUN  # Default to noun if POS tag is not recognized

    def preprocess(self, document):
        # Lowercase the text
        document = document.lower()

        # Remove punctuation
        document = re.sub(f"[{re.escape(string.punctuation)}]", "", document)

        # Tokenize the text
        tokens = nltk.word_tokenize(document)

        # Remove stopwords and lemmatize
        tokens = [self.lemmatizer.lemmatize(word, pos=self.get_word_pos(word)) for word in tokens if word not in self.stop_words]

        # Join tokens back into a text
        cleaned_text = " ".join(tokens)

        return cleaned_text


In [152]:
specific_row= sampled_dataset.where('doc_id == '+str(54857)).select('TEXT').first()

In [153]:
text_value = specific_row['TEXT']

In [154]:
print(text_value)

guinea worm disease is endemic in west africa in a field drug trial was conducted to compare effectiveness cost and sideeffects of thiabendazole and metronidazole in treating AS guinea worm disease and preventing latent worms from emerging a mass chemotherapy campaign was planned to follow the drug trial only of the treated patients expelled the worm completely and in of the cases worms continued to appear both drugs were equally unsatisfactory in their antihelminthic effect consequently our efforts to control guinea worm have shifted from chemotherapy to chemical control of cyclops and improvement of rural water supplies


guinea worm disease is endemic in west africa in a field drug trial was conducted to compare effectiveness cost and sideeffects of thiabendazole and metronidazole in treating AS guinea worm disease and preventing latent worms from emerging a mass chemotherapy campaign was planned to follow the drug trial only of the treated patients expelled the worm completely and in of the cases worms continued to appear both drugs were equally unsatisfactory in their antihelminthic effect consequently our efforts to control guinea worm have shifted from chemotherapy to chemical control of cyclops and improvement of rural water supplies

### Test without POS WORD

In [34]:
#text_preprocessor = TextPreprocessor()

#document_test = text_preprocessor.preprocess(text_value)

#print(document_test)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


guinea worm disease endemic west africa field drug trial conducted compare effectiveness cost sideeffects thiabendazole metronidazole treating guinea worm disease preventing latent worm emerging mass chemotherapy campaign planned follow drug trial treated patient expelled worm completely case worm continued appear drug equally unsatisfactory antihelminthic effect consequently effort control guinea worm shifted chemotherapy chemical control cyclops improvement rural water supply

### Text w/ POS Word

In [155]:
text_preprocessor = TextPreprocessor()

In [156]:
document_test = text_preprocessor.preprocess(text_value)

In [157]:
print(document_test)

guinea worm disease endemic west africa field drug trial conduct compare effectiveness cost sideeffects thiabendazole metronidazole treat guinea worm disease prevent latent worm emerge mass chemotherapy campaign plan follow drug trial treat patient expel worm completely case worm continued appear drug equally unsatisfactory antihelminthic effect consequently effort control guinea worm shift chemotherapy chemical control cyclops improvement rural water supply


guinea worm disease endemic west africa field drug trial conduct compare effectiveness cost sideeffects thiabendazole metronidazole treat guinea worm disease prevent latent worm emerge mass chemotherapy campaign plan follow drug trial treat patient expel worm completely case worm continued appear drug equally unsatisfactory antihelminthic effect consequently effort control guinea worm shift chemotherapy chemical control cyclops improvement rural water supply

# Intermediate test FAILED

still to do:
- hash family choice
- k choice (based on avg length docs)

In [91]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

In [96]:
class TextPreprocessor_map():
    def __init__(self):
        # Initialize NLTK resources
        nltk.download("punkt")
        nltk.download("stopwords")
        nltk.download("wordnet")
        self.stop_words = set(stopwords.words("english"))
        self.lemmatizer = WordNetLemmatizer()

    # get position of words to lemmatize, info needed
    def get_word_pos(self, word):
        tagged_words = nltk.pos_tag([word])
        pos_tag = tagged_words[0][1]

        if pos_tag.startswith('J'):
            return wordnet.ADJ
        elif pos_tag.startswith('V'):
            return wordnet.VERB
        elif pos_tag.startswith('N'):
            return wordnet.NOUN
        elif pos_tag.startswith('R'):
            return wordnet.ADV
        else:
            return wordnet.NOUN  # Default to noun if POS tag is not recognized

    def preprocess(self, Text_Doc_id):

        document = Text_Doc_id[0]
        doc_id = Text_Doc_id[1]

        # Lowercase the text
        document = document.lower()

        # Remove punctuation
        document = re.sub(f"[{re.escape(string.punctuation)}]", "", document)

        # Tokenize the text
        tokens = nltk.word_tokenize(document)

        # Remove stopwords and lemmatize
        tokens = [self.lemmatizer.lemmatize(word, pos=self.get_word_pos(word)) for word in tokens if word not in self.stop_words]

        # Join tokens back into a text
        cleaned_text = " ".join(tokens)

        return (doc_id, cleaned_text)


In [99]:
text_preprocessor_map = TextPreprocessor_map()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [100]:
preproc = sampled_dataset.rdd.map(text_preprocessor_map)

In [101]:
preproc_df = preproc.toDF()

Traceback (most recent call last):
  File "/content/spark-3.4.0-bin-hadoop3/python/pyspark/serializers.py", line 459, in dumps
    return cloudpickle.dumps(obj, pickle_protocol)
  File "/content/spark-3.4.0-bin-hadoop3/python/pyspark/cloudpickle/cloudpickle_fast.py", line 73, in dumps
    cp.dump(obj)
  File "/content/spark-3.4.0-bin-hadoop3/python/pyspark/cloudpickle/cloudpickle_fast.py", line 632, in dump
    return Pickler.dump(self, obj)
TypeError: cannot pickle '_thread.RLock' object


PicklingError: ignored

In [106]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [107]:
#from filename_without_extension import ClassName

from text_preprocessor import TextPreprocessor

ModuleNotFoundError: ignored

In [108]:
!ls

drive		 pretrain_subset	  spark-3.4.0-bin-hadoop3.tgz
full_data.csv	 sample_data		  spark-3.4.0-bin-hadoop3.tgz.1
medal-emnlp.zip  spark-3.4.0-bin-hadoop3


In [109]:
%cd '/content/drive/MyDrive'

/content/drive/MyDrive


In [121]:
from text_processor import TextPreprocessor

In [123]:
tp = TextPreprocessor()

NameError: ignored

## Class Hash Family

In [78]:
# ci sono degli import che non ho eseguito ma ho potuto comunque creare classe
# forse sono già in colab boh

import re, hashlib, math, time
from random import randint, seed

# però sono abbastanza certa che il seed mi serva

seed(16)

In [79]:
# HO MESSO BYTE RITORNO A 4, PENSO 2^32 BUCKET THEN
# BUT WHO KNOWS

class hashFamily:
    def __init__(self, i):
        self.resultSize = 4 # how many bytes we want back
        self.maxLen = 20 # how long can our i be (in decimal)
        self.salt = str(i).zfill(self.maxLen)[-self.maxLen:]
        self.id = i

    def get_hash_value(self, el_to_hash):
        return int(hashlib.sha1(str(el_to_hash).encode('utf-8') + self.salt.encode('utf-8')).hexdigest()[-self.resultSize:], 16)


## Class Shingler

le do il dataset con documento già preprocessato

In [112]:
class shingler:
    def __init__(self, k):

        if k > 0:
            self.k = int(k)
        else:
            self.k = 10

    #inner class utility
    #def process_doc(self, document):
     #   return re.sub("( )+|(\n)+"," ",document).lower()

    def get_shingles(self, document):
        shingles = set()

        text_preprocessor = TextPreprocessor()

        document= text_preprocessor.preprocess(document)
        for i in range(0, len(document)-self.k+1 ):
            shingles.add(document[i:i+self.k])
        return shingles

    def get_k(self):
        return self.k

    #return sorted hash
    def get_hashed_shingles(self, shingles_set):
        hash_function = hashFamily(0)
        return sorted( {hash_function.get_hash_value(s) for s in shingles_set} )

## Shingling

In [113]:
def shingling_map(row):
    out = []
    sh_instance = shingler(10)
    hashed_shingles = sh_instance.get_hashed_shingles( sh_instance.get_shingles( row['TEXT'] ) )
    signature_size = 50
    for i in range(0,signature_size): #signature size
        out.append( (row['doc_id'], hashed_shingles, i ) )
    #return an iterator to use flatMap => produce more than one key-value pair as output (namely one per hash function)
    return iter(out)




In [114]:
docId_shingleset_hfunc = sampled_dataset.rdd.flatMap(shingling_map)

In [115]:
docId_shingleset_hfunc.toDF().show()

Py4JJavaError: ignored

In [116]:
!ls

 Asset-Portfolio-Management-usingDeep-Reinforcement-Learning--main
 Candidate_trial.csv
 colab_data_download_analysis_and_forecasting_pt2.ipynb
'Colab Notebooks'
'entity res.ipynb'
 __pycache__
 text_processor.py
 Untitled0.ipynb
 Untitled1.ipynb


# Another TEST FAILED


In [158]:
# HASH FAMILY

import re, hashlib, math, time
from random import randint, seed

# però sono abbastanza certa che il seed mi serva

seed(16)

# HO MESSO BYTE RITORNO A 4, PENSO 2^32 BUCKET THEN
# BUT WHO KNOWS

class hashFamily:
    def __init__(self, i):
        self.resultSize = 4 # how many bytes we want back
        self.maxLen = 20 # how long can our i be (in decimal)
        self.salt = str(i).zfill(self.maxLen)[-self.maxLen:]
        self.id = i

    def get_hash_value(self, el_to_hash):
        return int(hashlib.sha1(str(el_to_hash).encode('utf-8') + self.salt.encode('utf-8')).hexdigest()[-self.resultSize:], 16)


In [159]:
my_processor = TextPreprocessor()

In [186]:
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words("english"))

In [187]:
# SHINGLER


class shingler():
    def __init__(self, k, lemmatizer, stop_words):

        self.stop_words = lemmatizer
        self.lemmatizer = stop_words

        if k > 0:
            self.k = int(k)
        else:
            self.k = 10

    #inner class utility

    #-------------------------------

    # get position of words to lemmatize, info needed
    def get_word_pos(self, word):
        tagged_words = nltk.pos_tag([word])
        pos_tag = tagged_words[0][1]

        if pos_tag.startswith('J'):
            return wordnet.ADJ
        elif pos_tag.startswith('V'):
            return wordnet.VERB
        elif pos_tag.startswith('N'):
            return wordnet.NOUN
        elif pos_tag.startswith('R'):
            return wordnet.ADV
        else:
            return wordnet.NOUN  # Default to noun if POS tag is not recognized

    def preprocess(self, document):
        # Lowercase the text
        document = document.lower()

        # Remove punctuation
        document = re.sub(f"[{re.escape(string.punctuation)}]", "", document)

        # Tokenize the text
        tokens = nltk.word_tokenize(document)

        # Remove stopwords and lemmatize
        tokens = [self.lemmatizer.lemmatize(word, pos=self.get_word_pos(word)) for word in tokens if word not in self.stop_words]

        # Join tokens back into a text
        cleaned_text = " ".join(tokens)

        return cleaned_text
    # --------------------------------------

    #def process_doc(self, document):
     #   return re.sub("( )+|(\n)+"," ",document).lower()

    def get_shingles(self, document):
        shingles = set()

        document= self.preprocess(document)
        for i in range(0, len(document)-self.k+1 ):
            shingles.add(document[i:i+self.k])
        return shingles

    def get_k(self):
        return self.k

    #return sorted hash
    def get_hashed_shingles(self, shingles_set):
        hash_function = hashFamily(0)
        return sorted( {hash_function.get_hash_value(s) for s in shingles_set} )

In [191]:
# SHINGLING
def shingling_map(row):
    out = []
    sh_instance = shingler(10, lemmatizer, stop_words)
    hashed_shingles = sh_instance.get_hashed_shingles( sh_instance.get_shingles( row['TEXT'] ) )
    signature_size = 50
    for i in range(0,signature_size): #signature size
        out.append( (row['doc_id'], hashed_shingles, i ) )
    #return an iterator to use flatMap => produce more than one key-value pair as output (namely one per hash function)
    return iter(out)




In [194]:
docId_shingleset_hfunc_2 = sampled_dataset.rdd.flatMap(shingling_map)

In [195]:
docId_shingleset_hfunc_2.toDF().show()

Traceback (most recent call last):
  File "/content/spark-3.4.0-bin-hadoop3/python/pyspark/serializers.py", line 459, in dumps
    return cloudpickle.dumps(obj, pickle_protocol)
  File "/content/spark-3.4.0-bin-hadoop3/python/pyspark/cloudpickle/cloudpickle_fast.py", line 73, in dumps
    cp.dump(obj)
  File "/content/spark-3.4.0-bin-hadoop3/python/pyspark/cloudpickle/cloudpickle_fast.py", line 632, in dump
    return Pickler.dump(self, obj)
TypeError: cannot pickle '_thread.RLock' object


PicklingError: ignored

In [198]:
from pyspark.sql import SparkSession
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re
import string





In [203]:
# Initialize NLTK resources inside the shingler class
class shingler:
    def __init__(self, k):
        nltk.download("punkt")
        nltk.download("stopwords")
        nltk.download("wordnet")
        self.lemmatizer = WordNetLemmatizer()
        self.stop_words = set(stopwords.words("english"))

        if k > 0:
            self.k = int(k)
        else:
            self.k = 10

    @staticmethod
    def preprocess(document):
        # Lowercase the text
        document = document.lower()

        # Remove punctuation
        document = re.sub(f"[{re.escape(string.punctuation)}]", "", document)

        # Tokenize the text
        tokens = nltk.word_tokenize(document)

        # Remove stopwords and lemmatize
        tokens = [self.lemmatizer.lemmatize(word, pos=shingler.get_word_pos(word)) for word in tokens if word not in self.stop_words]

        # Join tokens back into a text
        cleaned_text = " ".join(tokens)

        return cleaned_text

    def get_shingles(self, document):
        shingles = set()

        document= self.preprocess(document)
        for i in range(0, len(document)-self.k+1 ):
            shingles.add(document[i:i+self.k])
        return shingles

    def get_k(self):
        return self.k

    #return sorted hash
    def get_hashed_shingles(self, shingles_set):
        hash_function = hashFamily(0)
        return sorted( {hash_function.get_hash_value(s) for s in shingles_set} )


In [204]:



# Define the shingling_map function
def shingling_map(row):
    out = []
    sh_instance = shingler(k)
    hashed_shingles = sh_instance.get_hashed_shingles(sh_instance.get_shingles(row['TEXT']))
    signature_size = 50
    for i in range(0, signature_size):
        out.append((row['doc_id'], hashed_shingles, i))
    return iter(out)





In [205]:
# Apply the shingling_map function to the dataset
a = sampled_dataset.rdd.flatMap(shingling_map)


In [206]:

# Show the resulting DataFrame
a.toDF().show()

Py4JJavaError: ignored

# Yet another test SEEMS TO WORK :)))

In [207]:
!pip install spacy
!python -m spacy download en_core_web_sm


2023-09-11 20:41:04.891806: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-11 20:41:07.321076: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 46.8 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [210]:
# Import spaCy and download the English model
import spacy
import en_core_web_sm


In [211]:
# Load the spaCy English model
nlp = en_core_web_sm.load()

In [212]:
class TextPreprocessor:
    def __init__(self):
        # Initialize spaCy resources
        self.nlp = spacy.load("en_core_web_sm")

    def preprocess(self, document):
        # Apply spaCy NLP pipeline
        doc = self.nlp(document)

        # Lemmatize and remove stopwords
        tokens = [token.lemma_ for token in doc if not token.is_stop]

        # Join tokens back into a text
        cleaned_text = " ".join(tokens)

        return cleaned_text

In [213]:
specific_row= sampled_dataset.where('doc_id == '+str(54857)).select('TEXT').first()

In [214]:
text_value = specific_row['TEXT']

In [215]:
print(text_value)

guinea worm disease is endemic in west africa in a field drug trial was conducted to compare effectiveness cost and sideeffects of thiabendazole and metronidazole in treating AS guinea worm disease and preventing latent worms from emerging a mass chemotherapy campaign was planned to follow the drug trial only of the treated patients expelled the worm completely and in of the cases worms continued to appear both drugs were equally unsatisfactory in their antihelminthic effect consequently our efforts to control guinea worm have shifted from chemotherapy to chemical control of cyclops and improvement of rural water supplies


In [216]:
text_preprocessor_spacy = TextPreprocessor()

In [217]:
document_test_spacy = text_preprocessor.preprocess(text_value)

In [218]:
print(document_test_spacy)

guinea worm disease endemic west africa field drug trial conduct compare effectiveness cost sideeffects thiabendazole metronidazole treat guinea worm disease prevent latent worm emerge mass chemotherapy campaign plan follow drug trial treat patient expel worm completely case worm continued appear drug equally unsatisfactory antihelminthic effect consequently effort control guinea worm shift chemotherapy chemical control cyclops improvement rural water supply


In [219]:
# HASH FAMILY


# ci sono degli import che non ho eseguito ma ho potuto comunque creare classe
# forse sono già in colab boh

import re, hashlib, math, time
from random import randint, seed

# però sono abbastanza certa che il seed mi serva

seed(16)

# HO MESSO BYTE RITORNO A 4, PENSO 2^32 BUCKET THEN
# BUT WHO KNOWS

class hashFamily:
    def __init__(self, i):
        self.resultSize = 4 # how many bytes we want back
        self.maxLen = 20 # how long can our i be (in decimal)
        self.salt = str(i).zfill(self.maxLen)[-self.maxLen:]
        self.id = i

    def get_hash_value(self, el_to_hash):
        return int(hashlib.sha1(str(el_to_hash).encode('utf-8') + self.salt.encode('utf-8')).hexdigest()[-self.resultSize:], 16)


In [220]:
# SHINGLER


class shingler:
    def __init__(self, k):

        if k > 0:
            self.k = int(k)
        else:
            self.k = 10

    #inner class utility
    #def process_doc(self, document):
     #   return re.sub("( )+|(\n)+"," ",document).lower()

    def get_shingles(self, document):
        shingles = set()

        spacy_proc = TextPreprocessor()

        document= spacy_proc.preprocess(document)

        for i in range(0, len(document)-self.k+1 ):
            shingles.add(document[i:i+self.k])
        return shingles

    def get_k(self):
        return self.k

    #return sorted hash
    def get_hashed_shingles(self, shingles_set):
        hash_function = hashFamily(0)
        return sorted( {hash_function.get_hash_value(s) for s in shingles_set} )

In [221]:
def shingling_map(row):
    out = []
    sh_instance = shingler(10)
    hashed_shingles = sh_instance.get_hashed_shingles( sh_instance.get_shingles( row['TEXT'] ) )
    signature_size = 50
    for i in range(0,signature_size): #signature size
        out.append( (row['doc_id'], hashed_shingles, i ) )
    #return an iterator to use flatMap => produce more than one key-value pair as output (namely one per hash function)
    return iter(out)




In [222]:
rdd_0 = sampled_dataset.rdd.flatMap(shingling_map)

In [223]:
rdd_0.first()

(54857,
 [121,
  163,
  727,
  851,
  925,
  1194,
  1283,
  1349,
  1502,
  1554,
  1767,
  1804,
  1850,
  1989,
  2075,
  2250,
  2281,
  2337,
  2478,
  2498,
  2738,
  3033,
  3199,
  3423,
  3451,
  3514,
  3538,
  3559,
  3599,
  3663,
  3758,
  3936,
  4067,
  4216,
  4346,
  4410,
  4458,
  4462,
  4649,
  4671,
  4807,
  5058,
  5169,
  5244,
  5288,
  5375,
  5448,
  5759,
  5777,
  5813,
  6025,
  6028,
  6486,
  6628,
  6668,
  6775,
  6862,
  6954,
  7280,
  8478,
  8616,
  8963,
  8977,
  8991,
  9273,
  9521,
  9554,
  9763,
  10101,
  10742,
  10754,
  10876,
  10930,
  11139,
  11511,
  11615,
  11647,
  11679,
  11732,
  11757,
  11911,
  12331,
  12535,
  12728,
  12738,
  13077,
  13208,
  13298,
  13511,
  13909,
  13910,
  14103,
  14600,
  14707,
  15097,
  15131,
  15175,
  15233,
  15327,
  15406,
  15574,
  15680,
  15782,
  16243,
  16792,
  16840,
  16896,
  17785,
  18294,
  18513,
  18516,
  18578,
  18592,
  18748,
  19121,
  19594,
  19790,
  19915,
  2

In [225]:
rdd_0_df = rdd_0.toDF()

In [226]:
rdd_0_df.show()

+-----+--------------------+---+
|   _1|                  _2| _3|
+-----+--------------------+---+
|54857|[121, 163, 727, 8...|  0|
|54857|[121, 163, 727, 8...|  1|
|54857|[121, 163, 727, 8...|  2|
|54857|[121, 163, 727, 8...|  3|
|54857|[121, 163, 727, 8...|  4|
|54857|[121, 163, 727, 8...|  5|
|54857|[121, 163, 727, 8...|  6|
|54857|[121, 163, 727, 8...|  7|
|54857|[121, 163, 727, 8...|  8|
|54857|[121, 163, 727, 8...|  9|
|54857|[121, 163, 727, 8...| 10|
|54857|[121, 163, 727, 8...| 11|
|54857|[121, 163, 727, 8...| 12|
|54857|[121, 163, 727, 8...| 13|
|54857|[121, 163, 727, 8...| 14|
|54857|[121, 163, 727, 8...| 15|
|54857|[121, 163, 727, 8...| 16|
|54857|[121, 163, 727, 8...| 17|
|54857|[121, 163, 727, 8...| 18|
|54857|[121, 163, 727, 8...| 19|
+-----+--------------------+---+
only showing top 20 rows



In [229]:
rdd_0_df.isLocal()

False

In [230]:
rdd_0_df.count()

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "spark-3.4.0-bin-hadoop3/python/lib/py4j-0.10.9.7-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "spark-3.4.0-bin-hadoop3/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: ignored